In [65]:
import pip

In [66]:
pip.main(['install','/datasets/home/34/234/cs291gbl/fastText/. --user'])

Invalid requirement: '/datasets/home/34/234/cs291gbl/fastText/. --user'
It looks like a path. Does it exist ?
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


1

In [67]:
import numpy as np
import pip
import torch
from torch import nn
import argparse
from torch.autograd import Variable
import io
from scipy import linalg
# import torch
print(torch.__version__)

0.3.1.post3


In [68]:
def read_txt_embeddings(path, full_vocab=False):
    """
    Reload pretrained embeddings from a text file.
    """
    word2id = {}
    vectors = []
    id2word={}


    emb_path = path
    emb_dim_file = 300
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        for i, line in enumerate(f):
            if i == 0:
                split = line.split()
                assert len(split) == 2
                assert emb_dim_file == int(split[1])
            else:
                word, vect = line.rstrip().split(' ', 1)
                if not full_vocab:
                    word = word.lower()
                vect = np.fromstring(vect, sep=' ')
                if np.linalg.norm(vect) == 0:  # avoid to have null embeddings
                    vect[0] = 0.01

                assert vect.shape == (emb_dim_file,), i
                word2id[word] = len(word2id)
                id2word[len(word2id)]=word
                vectors.append(vect[None])
            if len(word2id) >= 200000 and not full_vocab:
                break

    assert len(word2id) == len(vectors)
    print("Loaded %i pre-trained word embeddings." % len(vectors))


    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.concatenate(vectors, 0)
    embeddings = torch.from_numpy(embeddings).float()
    embeddings = embeddings
    assert embeddings.size() == (200000, 300)
    return embeddings,word2id,id2word

#for loading fastText
def load_fasttext_model(path):
    """
    Load a binarized fastText model.
    """
    try:
        import fastText
    except ImportError:
        raise Exception("Unable to import fastText. Please install fastText for Python: "
                        "https://github.com/facebookresearch/fastText")
    return fastText.load_model(path)

In [69]:

# english=load_fasttext_model("data/wiki.en.bin")
# spanish=load_fasttext_model("data/wiki.es.bin")
src_embeddings,src_word2id,src_id2word=read_txt_embeddings("data/wiki.en.vec")
tgt_embeddings,tgt_word2id,tgt_id2word=read_txt_embeddings("data/wiki.es.vec")

Loaded 200000 pre-trained word embeddings.
Loaded 200000 pre-trained word embeddings.


In [70]:
def get_batch2(X,M):
    N = X.size()[0]
    N=200000
    batch_indices = torch.LongTensor( np.random.randint(0,N,size=M) )
    batch_xs = torch.index_select(X,0,batch_indices)
    #batch_ys = torch.index_select(Y,0,batch_indices)
    return Variable(batch_xs, requires_grad=False).cuda()

In [71]:
class Discriminator(nn.Module):

    def __init__(self, dis_hid_dim,dis_dropout,dis_input_dropout):
        super(Discriminator, self).__init__()

        self.emb_dim = 300
        self.dis_layers = 3
        self.dis_hid_dim = dis_hid_dim
        self.dis_dropout = dis_dropout
        self.dis_input_dropout = dis_input_dropout

        layers = [nn.Dropout(self.dis_input_dropout)]
        for i in range(self.dis_layers + 1):
            input_dim = self.emb_dim if i == 0 else self.dis_hid_dim
            output_dim = 1 if i == self.dis_layers else self.dis_hid_dim
            layers.append(nn.Linear(input_dim, output_dim))
            if i < self.dis_layers:
                layers.append(nn.LeakyReLU(0.2))
                layers.append(nn.Dropout(self.dis_dropout))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        assert x.dim() == 2 and x.size(1) == self.emb_dim
        return self.layers(x).view(-1)

In [72]:
def Orthogonal(mapping,beta):
    W=mapping.weight.data
    mapping.weight.data.copy_((1 + beta) * W - beta * W.mm(W.transpose(0, 1).mm(W)))
#     print(mapping.weight.data.transpose(0, 1).mm(W))
    return mapping


In [73]:
#set up linear mapping,
mapping = nn.Linear(300, 300, bias=False)
mapping.weight.data.copy_(torch.eye(300))
modelDisc= Discriminator(2048,0.3,0.1)
mapping.cuda()
modelDisc.cuda()

Discriminator(
  (layers): Sequential(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=300, out_features=2048, bias=True)
    (2): LeakyReLU(0.2)
    (3): Dropout(p=0.3)
    (4): Linear(in_features=2048, out_features=2048, bias=True)
    (5): LeakyReLU(0.2)
    (6): Dropout(p=0.3)
    (7): Linear(in_features=2048, out_features=2048, bias=True)
    (8): LeakyReLU(0.2)
    (9): Dropout(p=0.3)
    (10): Linear(in_features=2048, out_features=1, bias=True)
    (11): Sigmoid()
  )
)

In [74]:
def get_nn_avg_dist(emb, query, knn=10):
    """
    Compute the average distance of the knn nearest neighbors
    """
    bs = 1024
    all_distances = []
    emb = emb.transpose(0, 1).contiguous()
    for i in range(0, query.shape[0], bs):
        distances = query[i:i + bs].mm(emb)
        best_distances, _ = distances.topk(knn, dim=1, largest=True, sorted=True)
        all_distances.append(best_distances.mean(1).cpu())
    all_distances = torch.cat(all_distances)
    return all_distances.numpy()



In [75]:
def get_candidates(emb1,emb2):
    bs = 128

    all_scores = []
    all_targets = []

    # number of source words to consider
    n_src = emb1.size(0)
    n_src =15000
    knn = 10

        # average distances to k nearest neighbors
    average_dist1 = torch.from_numpy(get_nn_avg_dist(emb2, emb1, knn))
    average_dist2 = torch.from_numpy(get_nn_avg_dist(emb1, emb2, knn))
    average_dist1 = average_dist1.type_as(emb1)
    average_dist2 = average_dist2.type_as(emb2)

    # for every source word
    for i in range(0, n_src, bs):

        # compute target words scores
        scores = emb2.mm(emb1[i:min(n_src, i + bs)].transpose(0, 1)).transpose(0, 1)
        scores.mul_(2)
        scores.sub_(average_dist1[i:min(n_src, i + bs)][:, None] + average_dist2[None, :])
        best_scores, best_targets = scores.topk(2, dim=1, largest=True, sorted=True)

        # update scores / potential targets
        all_scores.append(best_scores.cpu())
        all_targets.append(best_targets.cpu())

    all_scores = torch.cat(all_scores, 0)
    all_targets = torch.cat(all_targets, 0)

    all_pairs = torch.cat([torch.arange(0, all_targets.size(0)).long().unsqueeze(1),all_targets[:, 0].unsqueeze(1)], 1)
    assert all_scores.size() == all_pairs.size() == (n_src, 2)

    # sort pairs by score confidence
    diff = all_scores[:, 0] - all_scores[:, 1]
    reordered = diff.sort(0, descending=True)[1]
    all_scores = all_scores[reordered]
    all_pairs = all_pairs[reordered]
    
    
    selected = all_pairs.max(1)[0] <= 15000
    mask = selected.unsqueeze(1).expand_as(all_scores).clone()
    all_scores = all_scores.masked_select(mask).view(-1, 2)
    all_pairs = all_pairs.masked_select(mask).view(-1, 2)
    return all_pairs


In [76]:

def create_dictionary(src_embedding,tgt_embedding):
    src2tgt = get_candidates(src_embedding,tgt_embedding)
    tgt2src = get_candidates(tgt_embedding,src_embedding)
    
    s2t_candidates = set([(a, b) for a, b in src2tgt])
    t2s_candidates = set([(a, b) for a, b in tgt2src])
#     print(s2t_candidates)
    final_pairs = s2t_candidates & t2s_candidates
    if len(final_pairs) == 0:
        print("Dictionary does not exist")
        return None
    dico = torch.LongTensor(list([[a, b] for (a, b) in final_pairs]))

    print('New train dictionary of %i pairs.' % dico.size(0))
    return dico.cuda()
    
        
# def build_dictionary(src_emb, tgt_emb, _params, s2t_candidates, t2s_candidates):
    

In [77]:
def dist_mean_cosine(src_emb, tgt_emb):
        """
        Mean-cosine model selection criterion.
        """
        
        # get normalized embeddings
        src_emb = mapping(Variable(src_emb, requires_grad=False).cuda())
        src_emb=torch.FloatTensor(src_emb.cpu().data.numpy()).cuda()
        tgt_emb = tgt_emb.cuda()
        src_emb = src_emb / src_emb.norm(2, 1, keepdim=True).expand_as(src_emb)
        tgt_emb = tgt_emb / tgt_emb.norm(2, 1, keepdim=True).expand_as(tgt_emb)
        

        dico = create_dictionary(src_emb, tgt_emb)
        dico_max_size = 10000

      
        if dico is None:
            mean_cosine = -1e9
        else:
            mean_cosine = (src_emb[dico[:dico_max_size, 0]] * tgt_emb[dico[:dico_max_size, 1]]).sum(1).mean()
        print("Mean cosine", mean_cosine)
            

In [78]:
linoptimizer = torch.optim.SGD(mapping.parameters(), lr=0.1)
discoptimizer= torch.optim.SGD(modelDisc.parameters(), lr=0.1)

loss_fn=torch.nn.BCELoss()

def get_xy():
    
    #get minibatch of spanish words
    spanish_batch=get_batch2(tgt_embeddings,batch_size)
    #getminibatch of english words
    english_batch=get_batch2(src_embeddings,batch_size)

    #generate fake spanish embeddings
    s_fake= mapping(english_batch)

    #stack real and fake
    x = torch.cat([spanish_batch, s_fake], 0)
    y = torch.FloatTensor(2 * batch_size).zero_()
    
    y[batch_size:] = 0.05
    y[:batch_size]=1-0.05
    y=Variable(y,requires_grad=False).cuda()
    return x,y

In [ ]:
epoch=100000
batch_size=32
# dog=Variable(src_embeddings[src_word2id['dog']]).cuda()

# perro=Variable(tgt_embeddings[tgt_word2id['perro']]).cuda()
for _ in range(epoch):
      
    
    modelDisc.train()
    mapping.eval()
    for __ in range(5):
        
        x,y=get_xy()
        y_pred = modelDisc(x)
        # Compute and print loss.
        

        loss= loss_fn(y_pred, y)

        discoptimizer.zero_grad()
       
        # Backward pass: compute gradient of the loss with respect to model
        # parameters
        loss.backward()
#         print(loss.data)

        # Calling the step function on an Optimizer makes an update to its
        # parameters
        discoptimizer.step()
        
        
    for __ in range(3):
        modelDisc.eval()
        mapping.train()
        x,y=get_xy()
        y=1-y
        y_pred = modelDisc(x)
        loss2= loss_fn(y_pred, y)
        linoptimizer.zero_grad()
        loss2.backward()
        linoptimizer.step()
        
        if _%1:
            mapping=Orthogonal(mapping,0.001)
        
    if _%10000==0:
        dist_mean_cosine(src_embeddings,tgt_embeddings)
#         cos = nn.CosineSimilarity(dim=0, eps=1e-6)
    #     print(type(perro),type(mapping(dog).cpu()))
#     print(cos(perro.cpu(),mapping(dog).cpu()).data)
    
    
    

Dictionary does not exist
('Mean cosine', -1000000000.0)
Dictionary does not exist
('Mean cosine', -1000000000.0)
Dictionary does not exist
('Mean cosine', -1000000000.0)
New train dictionary of 2 pairs.
('Mean cosine', 0.638550877571106)
New train dictionary of 7 pairs.
('Mean cosine', 0.601514995098114)
New train dictionary of 4 pairs.
('Mean cosine', 0.6637828946113586)


In [53]:
# # spanish_embeddings
# tgt_emb2=tgt_embeddings.cpu().numpy()
# # print(tgt_emd.shape)


def get_nn(word_emb, tgt_emb, tgt_id2word, K=5):
  
    scores = (tgt_emb / np.linalg.norm(tgt_emb, 2, 1)[:, None]).dot(word_emb / np.linalg.norm(word_emb))
    k_best = scores.argsort()[-K:][::-1]
    for i, idx in enumerate(k_best):
        print('%.4f - %s' % (scores[idx], tgt_id2word[idx]))

In [54]:
# def create_map(words,language):
#     id2word={}
#     for word in words:
#         id=language.get_word_id(word)
#         id2word[id]=word
#     return id2word
# spanish_map=create_map(Swords,spanish)
        

In [55]:
# cat_embedding=mapping(Variable(src_embeddings[src_word2id['same']],requires_grad=False).cuda()).cpu().data
# word_emd=cat_embedding.numpy()
# get_nn(word_emd,tgt_emb,tgt_id2word)


In [56]:
# cat_embedding=tgt_embeddings[tgt_word2id['misma']]
# word_emd=cat_embedding.numpy()
# get_nn(word_emd,fake_src_embeddings.cpu().numpy(),src_id2word)


In [57]:
fake_src_embeddings=(mapping(Variable(src_embeddings, requires_grad=False).cuda())).cuda()
tgt_embeddings=(tgt_embeddings).cuda()

In [58]:
fake_src_embeddings=torch.FloatTensor(fake_src_embeddings.cpu().data.numpy()).cuda()
# type(fake_src_embeddings)
# src_embedding=src_embeddings.cpu().detach().numpy()
# from sklearn.decomposition import PCA
# pca = PCA(n_components=2, whiten=True)  # TSNE(n_components=2, n_iter=3000, verbose=2)
# pca.fit(np.vstack([src_embedding, spanish_embeddings]))
# print('Variance explained: %.2f' % pca.explained_variance_ratio_.sum())
src_emb=fake_src_embeddings/fake_src_embeddings.norm(2, 1, keepdim=True).expand_as(fake_src_embeddings)
tgt_emb = tgt_embeddings / tgt_embeddings.norm(2, 1, keepdim=True).expand_as(tgt_embeddings)

In [59]:
# import matplotlib.pyplot as plt


# def plot_similar_word(src_words, src, src_emb, tgt_words, tgt, tgt_emb, pca):

#     Y = []
#     word_labels = []
#     for sw in src_words:
#         Y.append(src_emb[src.get_word_id(sw)])
#         word_labels.append(sw)
#     for tw in tgt_words:
#         Y.append(tgt_emb[tgt.get_word_id(tw)])
#         word_labels.append(tw)

#     # find tsne coords for 2 dimensions
#     Y = pca.transform(Y)
#     x_coords = Y[:, 0]
#     y_coords = Y[:, 1]

#     # display scatter plot
#     plt.figure(figsize=(10, 8), dpi=80)
#     plt.scatter(x_coords, y_coords, marker='x')

#     for k, (label, x, y) in enumerate(zip(word_labels, x_coords, y_coords)):
#         color = 'blue' if k < len(src_words) else 'red'  # src words in blue / tgt words in red
#         plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points', fontsize=19,
#                      color=color, weight='bold')

#     plt.xlim(x_coords.min() - 0.2, x_coords.max() + 0.2)
#     plt.ylim(y_coords.min() - 0.2, y_coords.max() + 0.2)
#     plt.title('Visualization of the multilingual word embedding space')

#     plt.show()
    

In [60]:
# src_words = ['dog', 'the', 'the', 'tennis', 'cat', 'conference']
# tgt_words = ['perro', 'el', 'la', u'tenis',  'gato', 'conferencia']

# plot_similar_word(src_words,english,english_embeddings.cpu().detach().numpy(),tgt_words,spanish,spanish_embeddings.cpu().detach().numpy(),pca)
# # english.get_word_id

In [61]:
dico=create_dictionary(src_emb,tgt_emb)
# print(type(src_embedding))

KeyboardInterrupt: 

In [39]:
 def procrustes(src_emb,tgt_emb):
        """
        Find the best orthogonal matrix mapping using the Orthogonal Procrustes problem
        https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem
        """
        A = src_emb[dico[:, 0]]
        B = tgt_emb[dico[:, 1]]
        W = mapping.weight.data
        M = B.transpose(0, 1).mm(A).cpu().numpy()
        U, S, V_t = linalg.svd(M, full_matrices=True)
        mapping.weight.data.copy_(torch.from_numpy(U.dot(V_t)).type_as(W))
#         return W

In [51]:
for i in range(5):
    procrustes(src_emb,tgt_emb)


-7.8339e-01  2.0606e-01 -1.1714e-01  ...  -1.0379e-02  6.0843e-03  2.0496e-02
-2.4279e-01  1.0654e-01 -3.6599e-03  ...  -2.8994e-02 -1.4005e-01 -2.3610e-02
 1.0801e-01  4.1441e-02  1.7987e-01  ...  -5.9630e-03 -1.2171e-02  1.8207e-02
                ...                   ⋱                   ...                
 3.1065e-02  1.6978e-02  2.2879e-02  ...   1.5515e-04 -6.9401e-02 -2.6943e-02
-2.1687e-02 -3.4335e-02  4.5958e-02  ...   8.4566e-03  1.0589e-02 -6.1813e-02
-1.6965e-03 -1.1995e-02 -1.7960e-02  ...   1.3065e-02  1.6446e-02 -5.5869e-02
[torch.cuda.FloatTensor of size 300x300 (GPU 0)]

In [52]:
fake_src_embeddings2=(mapping(Variable(src_embeddings, requires_grad=False).cuda())).cuda()

In [57]:
cat_embedding=mapping(Variable(src_embeddings[src_word2id['the']],requires_grad=False).cuda()).cpu().data
word_emd=cat_embedding.numpy()
get_nn(word_emd,tgt_embeddings.cpu().numpy(),tgt_id2word)

0.1912 - fake
0.1863 - dormal
0.1809 - christina
0.1776 - lung
0.1758 - alternativo
